**Information extraced from DeepLabv3+ model outpout**

**For landfill segmentation and identification, two key metrics are commonly used to evaluate model performance:**

1.Pixel Accuracy:
This metric reflects the overall accuracy of pixel-level classification. For example, if a pixel is labeled as part of a landfill in both the model’s prediction and the ground truth, it is counted as correct. For a 512×512 test image, there are 512×512 pixel-level judgments. Higher pixel accuracy indicates that more pixels are correctly classified.

2.IoU (Intersection over Union):
IoU measures the overlap between the predicted segmentation and the ground truth. It is defined as the area of overlap divided by the area of union. Compared to pixel accuracy, IoU places more emphasis on the actual object regions and less on background pixels.

**Below are the results obtained when the fine-tuned DeepLabv3+ model was applied to the test dataset, which consists of 288 images, each with a resolution of 512x512 pixels.**

In [2]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import accuracy_score, jaccard_score

# Paths to test dataset
TEST_IMAGES_DIR = 'dataset/test/images'
TEST_MASKS_DIR = 'dataset/test/masks'

# Load the saved model
model = tf.keras.models.load_model('deeplabv3_model.h5',
                                   custom_objects={'MeanIoU': tf.keras.metrics.MeanIoU})

# Parameters for the test set
IMG_HEIGHT, IMG_WIDTH = 512, 512


# Function to preprocess images and masks
def preprocess_image(image_path, target_size):
    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image) / 255.0  # Normalize to [0, 1]
    return np.expand_dims(image, axis=0)  # Add batch dimension


def preprocess_mask(mask_path, target_size):
    mask = load_img(mask_path, target_size=target_size, color_mode="grayscale")
    mask = img_to_array(mask) / 255.0  # Normalize to [0, 1]
    mask = np.where(mask > 0.5, 1, 0)  # Binarize
    return mask.squeeze()  # Remove single channel dimension


# Evaluate on the test set
def evaluate_model(model, test_images_dir, test_masks_dir):
    image_filenames = os.listdir(test_images_dir)
    all_predictions = []
    all_ground_truths = []

    for file_name in image_filenames:
        # Preprocess image and mask
        image_path = os.path.join(test_images_dir, file_name)
        mask_path = os.path.join(test_masks_dir, file_name.replace('.jpg', '.png'))

        image = preprocess_image(image_path, (IMG_HEIGHT, IMG_WIDTH))
        ground_truth = preprocess_mask(mask_path, (IMG_HEIGHT, IMG_WIDTH))

        # Predict mask
        prediction = model.predict(image)
        prediction = (prediction.squeeze() > 0.5).astype(np.uint8)  # Threshold prediction

        all_predictions.append(prediction.flatten())
        all_ground_truths.append(ground_truth.flatten())

    # Flatten all predictions and ground truths
    all_predictions = np.concatenate(all_predictions)
    all_ground_truths = np.concatenate(all_ground_truths)

    # Compute accuracy and IoU
    accuracy = accuracy_score(all_ground_truths, all_predictions)
    iou = jaccard_score(all_ground_truths, all_predictions, average='binary')

    print(f"Accuracy: {accuracy:.4f}")
    print(f"IoU: {iou:.4f}")


# Run evaluation
evaluate_model(model, TEST_IMAGES_DIR, TEST_MASKS_DIR)


1/1 [==============================] - 0s 25ms/step
Accuracy: 0.9503
IoU: 0.6855
